In [1]:
import librosa
import librosa.display
import os

import warnings
warnings.filterwarnings("ignore")

path_to_voice_records = "D:\\Utilisateurs\\ENAC\\projet_AIRBUS\\voice\\84\\121123\\"
path_to_airplane_records = "D:\\Utilisateurs\\ENAC\\projet_AIRBUS\\recordings\\"
path_to_created_samples = "D:\\Utilisateurs\\ENAC\\projet_AIRBUS\\created\\"

l_voice_records = os.listdir(path_to_voice_records)
l_airplane_records = os.listdir(path_to_airplane_records)
l_rec = l_voice_records + l_airplane_records

In [2]:
def mat_to_list(mat):
    out = []
    for x in mat:
        l_x = []
        for i in range(len(x)):
            l_x.append(x[i])
        out.append(l_x)
    return out

In [21]:
import random as rd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import numpy as np
n_train = 2

sp_voice = [] #spectres 
sp_plane = []
pca = PCA(0.99)
pca_xtrain = None

for i in range(100): 
    picked = rd.choice(l_voice_records) #pour l'instant que sur les voix
    signal,sample_rate = librosa.load(path_to_voice_records+picked)
    spectrum = librosa.stft(y=signal,center=False) #tronqué car les voix n'ont pas le meme temps d'enregistrement
    spectrum = spectrum.astype(float)
    sp_voice.append(spectrum)

for i in range(4): 
    picked = rd.choice(l_airplane_records) 
    signal,sample_rate = librosa.load(path_to_airplane_records+picked)
    spectrum = librosa.stft(y=signal,center=False) #tronqué car les voix n'ont pas le meme temps d'enregistrement

    #On rescale la data
    spectrum = spectrum.astype(float)
    sp_plane.append(spectrum)



In [22]:
X_voice= np.transpose(np.concatenate(sp_voice,axis=1))
X_plane = np.transpose(np.concatenate(sp_plane,axis=1))

In [23]:
len(X_voice),len(X_voice[0])

(23596, 1025)

In [24]:
# len(sp_plane)

# for i in range(n_train):
#     sp_voice[i] = pca.transform(sp_voice[i])
#     sp_plane[i] = pca.transform(sp_plane[i])

In [25]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

y_voice = [1 if max(elt)>1 else -1 for elt in X_voice]
y_plane = [0 if max(elt)>1 else -1 for elt in X_plane]

X = np.concatenate([X_voice,X_plane])
# X = PCA(0.999).fit_transform(X)
y = y_voice + y_plane

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.33, random_state=42)

rf = RandomForestClassifier(n_jobs=-1)
rf.fit(X_train,y_train)

RandomForestClassifier(n_jobs=-1)

In [26]:
from sklearn.metrics import accuracy_score

pred = rf.predict(X_test)
accuracy_score(pred,y_test)

0.9888164520061483